In [1]:
import httpx
import geopandas as gp
from shapely.geometry import Point
import pandas as pd

## Stations

In [2]:
mode_meta_query = httpx.get("https://api.tfl.gov.uk/Line/Meta/Modes")

In [3]:
modes = [mode['modeName'] for mode in mode_meta_query.json() if mode['isScheduledService']]
modes.remove('bus')
modes.remove('coach')

In [4]:
stops = []
for mode in modes:
    mode_status_query = httpx.get(f'https://api.tfl.gov.uk/line/mode/{mode}/status')
    lines = {line['id']: line['name'] for line in mode_status_query.json() if line['modeName']==mode}
    for line_id, line_name in lines.items():
        lines_info = httpx.get(f'https://api.tfl.gov.uk/line/{line_id}/stoppoints')
        result = []
        for line_info in lines_info.json():  
            try:
                result_ = dict(Name=line_info['commonName'], mode=mode, line=line_name, geometry=Point(line_info['lon'], line_info['lat'])) 
            except:
                a = lines_info.json()    
            result.append(result_)
        stops.extend(result)    

In [8]:
a

{'statusCode': 429,
 'message': 'Rate limit is exceeded. Try again in 48 seconds.'}

In [6]:
stations = gp.GeoDataFrame(stops, crs="EPSG:4326", geometry='geometry')
stations['line'] = stations.groupby('Name')['line'].transform(', '.join)
stations = stations.drop_duplicates()

In [7]:
stations

,Name,mode,line,geometry
0,Emirates Greenwich Peninsula,cable-car,Emirates Air Line,POINT (0.00834 51.49957)
1,Emirates Royal Docks,cable-car,Emirates Air Line,POINT (0.01765 51.50773)
2,Abbey Road DLR Station,dlr,DLR,POINT (0.00374 51.53193)
3,All Saints DLR Station,dlr,DLR,POINT (-0.01314 51.51100)
4,Beckton DLR Station,dlr,DLR,POINT (0.06145 51.51436)
...,...,...,...,...
3766,Tower Hill Underground Station,tube,Circle,POINT (-0.07655 51.50997)
3767,Victoria Underground Station,tube,Circle,POINT (-0.14310 51.49636)
3768,Wood Lane Underground Station,tube,Circle,POINT (-0.22453 51.50967)
3769,Westminster Underground Station,tube,Circle,POINT (-0.12486 51.50132)


## Air Quality

In [24]:
latest_london_air_quality = httpx.get("https://api.erg.ic.ac.uk/AirQuality/Daily/MonitoringIndex/Latest/GroupName=London/Json", timeout=30)

In [25]:
local_authority_air_quality = latest_london_air_quality.json()['DailyAirQualityIndex']['LocalAuthority']

In [26]:
def parse_data_from_sites(result: dict, site: dict, species, local_authority: dict):
    site_name = f"{site['@SiteCode']}{species['@SpeciesCode']}"
    result[site_name] = dict(
        SiteName=site['@SiteName'],
        LocalAuthorityName=local_authority['@LocalAuthorityName'],
        Latitude=site['@Latitude'],
        Longitude=site['@Longitude'],
        Species=str(dict(SpeciesCode=species['@SpeciesCode'],
        SpeciesDescription=species['@SpeciesDescription'],
        AirQualityIndex=species['@AirQualityIndex'],
        AirQualityBand=species['@AirQualityBand']))
    )
    return result

In [27]:
def parse_data_from_site(result, site, local_authority):
    species = site['Species']
    if isinstance(species, list):
        for single_species in species:
            result = parse_data_from_sites(result, site, single_species, local_authority)
    elif isinstance(species, dict):
        result = parse_data_from_sites(result, site, species, local_authority)
    return result

In [28]:
result_dict = dict()

for local_authority_data in local_authority_air_quality:
    if 'Site' in local_authority_data:
        sites_data = local_authority_data['Site']
        if isinstance(sites_data, list):
            for site_data in sites_data:
                parse_data_from_site(result_dict, site_data, local_authority_data)
        elif isinstance(sites_data, dict):
                parse_data_from_site(result_dict, sites_data, local_authority_data)

In [29]:
air_quality_df = pd.DataFrame.from_dict(result_dict).T

In [30]:
air_quality_df['Species'] = air_quality_df.groupby('SiteName')['Species'].transform(' '.join)


In [31]:
air_quality_df

,SiteName,LocalAuthorityName,Latitude,Longitude,Species
BG1NO2,Barking and Dagenham - Rush Green,Barking and Dagenham,51.563752,0.177891,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BG1SO2,Barking and Dagenham - Rush Green,Barking and Dagenham,51.563752,0.177891,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BQ7NO2,Bexley - Belvedere West,Bexley,51.4946486813055,0.137279111232178,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BQ7O3,Bexley - Belvedere West,Bexley,51.4946486813055,0.137279111232178,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BQ7PM10,Bexley - Belvedere West,Bexley,51.4946486813055,0.137279111232178,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
...,...,...,...,...,...
WM6PM10,Westminster - Oxford Street,Westminster,51.5139287404213,-0.152792701881935,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMBNO2,Westminster - Oxford Street East,Westminster,51.516066,-0.13516388,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMBPM10,Westminster - Oxford Street East,Westminster,51.516066,-0.13516388,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMCNO2,Westminster - Cavendish Square,Westminster,51.5168016452062,-0.145657269364411,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."


In [32]:
air_quality_df = air_quality_df.groupby('SiteName').first()

In [33]:
air_quality = gp.GeoDataFrame(air_quality_df, crs="EPSG:4326", 
                              geometry=gp.points_from_xy(air_quality_df['Longitude'], air_quality_df['Latitude']))

In [34]:
air_quality.tail()['Species'][0]

"{'SpeciesCode': 'NO2', 'SpeciesDescription': 'Nitrogen Dioxide', 'AirQualityIndex': '1', 'AirQualityBand': 'Low'}"

## Map

In [35]:
from keplergl import KeplerGl
kepler_map = KeplerGl()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [36]:
kepler_map.add_data(data=stations, name='TfL Stations')

In [37]:
kepler_map.add_data(data=air_quality, name='Air Quality')

In [38]:
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 51.5064,
            'longitude': -0.1,
            'zoom': 9.32053899007826
        }
    }
}

In [39]:
kepler_map.config = config

In [40]:
kepler_map

KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 51.5064, 'longitude': -0.1, 'zoom': 9.32…